# Collecting Anomalies

In [2]:
import feedparser
import pprint
from datetime import datetime, timedelta, date
import pandas as pd
from tqdm import tqdm
import requests
import time

In [3]:
url = "http://www.nasdaqtrader.com/rss.aspx?feed=tradehalts&haltdate={mmddyyyy}"
def get_halt_data(start_date: datetime.date, end_date: datetime.date, halt_codes: list) -> pd.DataFrame:
    # create empty data structure
    data_lst = []
    
    # Calculate the number of days in the range
    num_days = (end_date - start_date).days + 1
    
    # Loop through the date range with a progress bar
    for i in tqdm(range(num_days), desc="Processing dates"):
        current_date = start_date + timedelta(days=i)

        if i % 100 == 0:
            time.sleep(2)
        
        # parse URL
        str_date = current_date.strftime("%m%d%Y")
        formatted_url = url.format(mmddyyyy=str_date)

        try:
            # Fetch the feed with a timeout
            response = requests.get(formatted_url, timeout=10)
            response.raise_for_status()  # Raise an error for bad status codes
        
            # Parse the feed content
            feed = feedparser.parse(response.content)
        except requests.exceptions.Timeout:
            print('The request timed ou, current_date =', str_date)
            return pd.DataFrame(data_lst)
        except requests.exceptions.RequestException as e:
            print(f'An error occurred: {e}, current_date', str_date)
            return pd.DataFrame(data_lst)

        # check if there are any actual entries
        if len(feed.entries) > 0:
            # iterate through feed entries (articles)
            for entry in feed.entries:
                # only look for specified halt codes
                # continue otherwise
                if halt_codes is not None and entry.ndaq_reasoncode not in halt_codes:
                    continue 
                    
                # empty data entry
                data_entry = {}
                data_entry["halt_date"] = entry.ndaq_haltdate
                data_entry["resumption_date"] = entry.ndaq_resumptiondate
                data_entry["symbol"] = entry.title
                data_entry["issue_name"] = entry.ndaq_issuename
                data_entry["mkt_center"] = entry.ndaq_mkt
                data_entry["halt_code"] = entry.ndaq_reasoncode
                data_entry["halt_time"] = entry.ndaq_halttime
                data_entry["resumption_quote_time"] = entry.ndaq_resumptionquotetime
                data_entry["resumption_trade_time"] = entry.ndaq_resumptiontradetime

                # append to list
                data_lst.append(data_entry)
    
    return pd.DataFrame(data_lst)

Dates already covered: ``date(2015, 1, 1)`` to ``date(2025, 1, 15)``

In [ ]:
halt_df = get_halt_data(date(2024, 6, 19), date(2025, 1, 15), None)

In [ ]:
halt_df

In [ ]:
#halt_df.to_csv("test_data/halt_records.csv", index=False)
halt_df.to_csv("test_data/halt_records.csv", mode='a', index=False, header=False)
#halt_df = pd.read_csv("test_data/halt_records.csv")

In [4]:
halt_df = pd.read_csv("test_data/halt_records.csv")

In [5]:
halt_df

,halt_date,resumption_date,symbol,issue_name,mkt_center,halt_code,halt_time,resumption_quote_time,resumption_trade_time
0,01/02/2015,01/02/2015,PWRD,Perfect World Co Ltd ADS,Q,T3,07:58:48,08:50:00,08:55:00
1,01/02/2015,01/05/2015,FFKY,First Financial Service Corp,Q,D,08:13:17,00:00:01,00:00:01
2,01/02/2015,01/02/2015,BARL,Morgan Stanley S&P 500 Crude Oil Linked ETNs d...,P,M,09:30:18,09:36:00,09:36:00
3,01/02/2015,01/02/2015,RGRC,RBS Rogers Enhanced Commodity ETN,P,M,09:30:38,09:36:00,09:36:00
4,01/02/2015,01/02/2015,SMACU,Sino Mercury Acquisition Unit,Q,LUDP,09:30:49,09:30:49,09:35:49
...,...,...,...,...,...,...,...,...,...
96801,01/15/2025,01/15/2025,XRTX,XORTX Therapeutics Inc. Cm,Q,LUDP,13:46:52,13:46:52,13:51:52
96802,01/15/2025,01/15/2025,PTLE,PTL LTD Ordinary Shares,Q,LUDP,14:02:40,14:02:40,14:07:40
96803,01/15/2025,01/15/2025,PTLE,PTL LTD Ordinary Shares,Q,LUDP,15:29:16,15:29:16,15:34:16
96804,01/15/2025,01/16/2025,MBIO,"Mustang Bio, Inc. CM",Q,T3,19:50:00,08:55:00,09:00:00


In [6]:
halt_df.halt_code.unique()

array(['T3', 'D', 'M', 'LUDP', 'T2', 'T12', 'T1', 'H11', 'T7'],
      dtype=object)

In [7]:
halt_df[halt_df.halt_code == 'H11']

,halt_date,resumption_date,symbol,issue_name,mkt_center,halt_code,halt_time,resumption_quote_time,resumption_trade_time
57911,03/04/2022,03/14/2022,RUSL,Direxion Daily Russia Bull 2X Shares,P,H11,03:54:19,00:00:01,00:00:01
91893,08/07/2024,08/19/2024,SCPX,"Scorpius Holdings, Inc.",A,H11,09:53:26,09:35:00,09:35:00
93288,10/04/2024,10/15/2024,UAVS,Ageagle Aerial Systems Inc,N,H11,13:51:59,09:00:00,09:00:00


In [8]:
nsdq_sorted = halt_df[["halt_date", "symbol", "halt_code"]].groupby(["halt_date", "symbol", "halt_code"]).value_counts().reset_index().sort_values("count", ascending=False)

In [9]:
nsdq_sorted['halt_date'] = pd.to_datetime(nsdq_sorted['halt_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

In [10]:
nsdq_sorted.sort_values("halt_date")

,halt_date,symbol,halt_code,count
156,2012-01-03,RVSN,T3,1
154,2012-01-03,APKT,T3,1
155,2012-01-03,OSH,T7,2
329,2012-01-04,TRID,T3,1
328,2012-01-04,SATC,T3,1
...,...,...,...,...
2326,2025-01-15,MYNZ,LUDP,2
2323,2025-01-15,JXG,LUDP,4
2330,2025-01-15,SONM,LUDP,4
2322,2025-01-15,ALVR,T3,1


In [11]:
nsdq_sorted[nsdq_sorted["count"] >= 5].sort_values("count", ascending=False)[:50]

,halt_date,symbol,halt_code,count
11313,2020-03-12,AMCIU,LUDP,79
27419,2016-06-02,TRTLU,LUDP,70
28018,2016-06-06,TRTLU,LUDP,70
27572,2016-06-03,TRTLU,LUDP,70
42634,2015-09-15,SMACU,LUDP,69
27179,2015-06-01,FNTC,LUDP,69
42603,2015-09-15,AAPC,LUDP,69
31298,2016-06-27,OACQU,LUDP,69
29777,2015-06-17,FNTC,LUDP,67
721,2016-01-06,FNTC,LUDP,67


### NYSE Trade Halt Data

In [12]:
nyse_halts = pd.read_csv("test_data/Trade_halts_Historical.csv").rename(columns={'Halt Date': 'halt_date', 
                                                                                 'Symbol': 'symbol',
                                                                                 'Reason': 'halt_code'})

In [13]:
nyse_sorted = nyse_halts[["halt_date", "symbol", "halt_code"]].groupby(["halt_date", "symbol", "halt_code"]).value_counts().reset_index().sort_values("count", ascending=False)

In [14]:
nyse_sorted.halt_code.unique()

array(['LULD pause', 'News pending', 'Regulatory Concern',
       'Intraday Indicative Value Not Available', 'Corporate Action',
       'News Released', 'Merger Effective', 'Intraday Ind Val NA',
       'New Security Offering', 'News dissemination'], dtype=object)

In [15]:
nyse_sorted[nyse_sorted["count"] > 5].sort_values("count", ascending=False)[:50]

,halt_date,symbol,halt_code,count
1094,2020-02-10,FMCIU,LULD pause,60
7426,2020-06-08,HVT A,LULD pause,59
1874,2020-03-12,AMCIU,LULD pause,53
26942,2024-12-23,DRCT,LULD pause,49
17990,2023-03-13,WAL,LULD pause,46
20396,2023-10-11,TPST,LULD pause,46
16672,2022-12-09,AMAM,LULD pause,44
5199,2020-03-24,IMAC,LULD pause,42
26882,2024-12-19,NVNI,LULD pause,41
26864,2024-12-18,XCH,LULD pause,41


### Combining the Two Halt Datasets

In [23]:
stacked_df = pd.concat([nsdq_sorted, nyse_sorted], ignore_index=True)

In [24]:
stacked_dedup = stacked_df.drop_duplicates(subset=['halt_date', 'symbol'], keep='first').sort_values('halt_date').reset_index(drop=True)

In [115]:
stacked_dedup.to_csv('stacked_dedup_halts.csv')

**TODO:** Filter out irrelevant halt codes.

## Query WRDS database

There will be multiple steps to this. We will want 
1. The opening and closing price 15 days before and after the date of the LULD pause
2. The opening and closing price the day of the LULD pause
3. The trade price that triggered the LULD pause (last step)

We can find interday anomalies using 1 and 2. However, for intraday anomalies we will need to determine 3 and possibly some other metric for quantifying average price throughout the trade day.

Since interday anomalies will likely be easier to determine, we will develop the necessary queries for those first.

#### Preliminaries and Test Data

In [25]:
# imports
import pandas_market_calendars as mcal
import wrds

In [26]:
db = wrds.Connection(wrds_username='audreymcmillion')

Loading library list...
Done


In [27]:
## FUNCTIONS TO GET DAYS BEFORE AND AFTER A GIVEN TRADING DATE ##
nyse = mcal.get_calendar("NYSE")

def get_before_date(current_date, days_before):
    # get set of trading days
    trading_days = nyse.valid_days(start_date="2000-01-01", end_date=current_date)

    # Find the trade date days_before days before today
    trade_date_days_ago = trading_days[-1 - days_before]

    return trade_date_days_ago.strftime('%Y-%m-%d')

def get_after_date(current_date, days_after):
    today = pd.Timestamp.today().normalize()

    # get set of trading days
    trading_days = nyse.valid_days(start_date=current_date, end_date=today)

    # Find the trade date days_after days after today
    trade_date_days_after = trading_days[days_after]

    return trade_date_days_after.strftime('%Y-%m-%d')

In [28]:
get_after_date('2024-07-08', 15)

'2024-07-29'

Use the above to get the details.

In [109]:
# query for before and after average prices
before_after_query = """
    WITH price_details AS (
        SELECT  b.ticker,
                a.permno,
                a.dlycaldt,
                a.dlyopen,
                coalesce(a.dlyclose, a.dlyprc) AS dlyclose,
                a.dlyhigh,
                a.dlylow,
                a.dlynumtrd,
                a.dlyvol
        FROM crsp.dsf_v2 a
        JOIN crsp.dsenames AS b
        ON a.permno = b.permno
        AND b.ticker IN ({symbol_lst})
        AND a.dlycaldt BETWEEN date('{start_dt}') AND date('{end_dt}')
        AND date('{current_dt}') BETWEEN b.namedt AND b.nameendt
    ),

    days_before AS (
        SELECT ticker,
               MIN(dlycaldt) AS before_date,
               AVG(dlyopen) AS before_avg_open,
               AVG(dlyclose) AS before_avg_close,
               AVG(dlyhigh) AS before_avg_dlyhigh,
               AVG(dlylow) AS before_avg_dlylow,
               AVG(dlynumtrd) AS before_avg_dlynumtrd,
               AVG(dlyvol) AS before_avg_dlyvol
        FROM price_details
        WHERE dlycaldt BETWEEN date('{start_dt}') AND (date('{current_dt}') - INTERVAL '1 day')
        GROUP BY ticker
    ),

    current_days AS (
        SELECT ticker,
               MIN(dlycaldt) AS current_date,
               AVG(dlyopen) AS open_pr,
               AVG(dlyclose) AS close_pr,
               AVG(dlyhigh) AS dlyhigh,
               AVG(dlylow) AS dlylow,
               AVG(dlynumtrd) AS dlynumtrd,
               AVG(dlyvol) AS dlyvol
        FROM price_details
        WHERE dlycaldt = date('{current_dt}')
        GROUP BY ticker
    ),

    days_after AS (
        SELECT ticker,
               MAX(dlycaldt) AS after_date,
               AVG(dlyopen) AS after_avg_open,
               AVG(dlyclose) AS after_avg_close,
               AVG(dlyhigh) AS after_avg_dlyhigh,
               AVG(dlylow) AS after_avg_dlylow,
               AVG(dlynumtrd) AS after_avg_dlynumtrd,
               AVG(dlyvol) AS after_avg_dlyvol
        FROM price_details
        WHERE dlycaldt BETWEEN (date('{current_dt}') + INTERVAL '1 day') AND date('{end_dt}')
        GROUP BY ticker
    )

    SELECT coalesce(c.ticker, a.ticker, b.ticker) AS ticker,
           c.current_date,
           COALESCE(b.before_date, TO_DATE('{start_dt}', 'YYY-MM-DD')) AS before_date,
           COALESCE(a.after_date, TO_DATE('{end_dt}', 'YYYY-MM-DD')) AS after_date,
           c.open_pr,
           c.close_pr,
           c.dlyhigh,
           c.dlylow,
           c.dlynumtrd,
           c.dlyvol,
           b.before_avg_open,
           b.before_avg_close,
           b.before_avg_dlyhigh,
           b.before_avg_dlylow,
           b.before_avg_dlynumtrd,
           b.before_avg_dlyvol,
           a.after_avg_open,
           a.after_avg_close,
           a.after_avg_dlyhigh,
           a.after_avg_dlylow,
           a.after_avg_dlynumtrd,
           a.after_avg_dlyvol
    FROM current_days c
    FULL OUTER JOIN days_before b
    ON (c.ticker = b.ticker)
    FULL OUTER JOIN days_after a
    ON (c.ticker = a.ticker)
"""

In [110]:
max_date = datetime.strptime('2023-12-29', '%Y-%m-%d')
output_file = "before_after_results.csv"
i = 0
for current_date in tqdm(stacked_dedup.halt_date.unique()):
    print(current_date)

    # check if valid date
    if datetime.strptime(current_date, '%Y-%m-%d') >= max_date:
        continue
 
    # get subset of LULP 
    subset = stacked_dedup[(stacked_dedup.halt_date == current_date) \
                           & ((stacked_dedup.halt_code == 'LUDP') | (stacked_dedup.halt_code == 'LULD pause')) ]

    # get relevant symbols
    subset_symbols = subset.symbol.to_list()

    if len(subset_symbols) == 0:
        continue

    # get string version of symbols
    stringed_symbols = "', '".join(subset_symbols)
    stringed_symbols = f"'{stringed_symbols}'".replace("''", "'")
    

    # get date strings
    before_date = get_before_date(current_date, 15)
    after_date = get_after_date(current_date, 15)

    # check if valid date
    if datetime.strptime(after_date, '%Y-%m-%d') >= max_date:
        continue
    
    # query using extracted values
    before_after_result = db.raw_sql(before_after_query.format(start_dt = before_date,
                                     current_dt = current_date,
                                     end_dt = after_date,
                                     symbol_lst = stringed_symbols))

    # write to CSV
    if i == 0:
        # Write the header in the first iteration
        before_after_result.to_csv(output_file, mode='w', index=False)
    else:
        # Append without writing the header in subsequent iterations
        before_after_result.to_csv(output_file, mode='a', index=False, header=False)
        
    i += 1

  0%|▎                                                                                      | 12/3269 [00:00<00:28, 115.51it/s]

2012-01-03
2012-01-04
2012-01-05
2012-01-06
2012-01-09
2012-01-10
2012-01-11
2012-01-12
2012-01-13
2012-01-17
2012-01-18
2012-01-19
2012-01-20
2012-01-23
2012-01-24
2012-01-25
2012-01-26
2012-01-27
2012-01-30
2012-01-31


  1%|▉                                                                                       | 37/3269 [00:00<00:33, 97.17it/s]

2012-02-01
2012-02-02
2012-02-03
2012-02-06
2012-02-07
2012-02-08
2012-02-09
2012-02-10
2012-02-14
2012-02-15
2012-02-16
2012-02-17
2012-02-21
2012-02-22
2012-02-23
2012-02-27
2012-02-28
2012-02-29
2012-03-01
2012-03-02
2012-03-05
2012-03-06
2012-03-07
2012-03-08
2012-03-09


  2%|█▊                                                                                     | 66/3269 [00:00<00:26, 122.20it/s]

...

2023-12-18
2023-12-19
2023-12-20


 92%|██████████████████████████████████████████████████████████████████████████████▉       | 3001/3269 [05:45<00:20, 12.77it/s]

2023-12-21
2023-12-22
2023-12-26


100%|██████████████████████████████████████████████████████████████████████████████████████| 3269/3269 [05:45<00:00,  9.45it/s]

2023-12-27
2023-12-28
2023-12-29
2024-01-02
2024-01-03
2024-01-04
2024-01-05
2024-01-08
2024-01-09
2024-01-10
2024-01-11
2024-01-12
2024-01-16
2024-01-17
2024-01-18
2024-01-19
2024-01-22
2024-01-23
2024-01-24
2024-01-25
2024-01-26
2024-01-29
2024-01-30
2024-01-31
2024-02-01
2024-02-02
2024-02-05
2024-02-06
2024-02-07
2024-02-08
2024-02-09
2024-02-12
2024-02-13
2024-02-14
2024-02-15
2024-02-16
2024-02-20
2024-02-21
2024-02-22
2024-02-23
2024-02-26
2024-02-27
2024-02-28
2024-02-29
2024-03-01
2024-03-04
2024-03-05
2024-03-06
2024-03-07
2024-03-08
2024-03-11
2024-03-12
2024-03-13
2024-03-14
2024-03-15
2024-03-18
2024-03-19
2024-03-20
2024-03-21
2024-03-22
2024-03-25
2024-03-26
2024-03-27
2024-03-28
2024-04-01
2024-04-02
2024-04-03
2024-04-04
2024-04-05
2024-04-08
2024-04-09
2024-04-10
2024-04-11
2024-04-12
2024-04-15
2024-04-16
2024-04-17
2024-04-18
2024-04-19
2024-04-22
2024-04-23
2024-04-24
2024-04-25
2024-04-26
2024-04-29
2024-04-30
2024-05-01
2024-05-02
2024-05-03
2024-05-06
2024-05-07

#### Read in Results

In [141]:
results = pd.read_csv("before_after_results.csv")

In [142]:
nona_results = results.dropna().reset_index(drop=True)

In [143]:
nona_results

,ticker,current_date,before_date,after_date,open_pr,close_pr,dlyhigh,dlylow,dlynumtrd,dlyvol,...,before_avg_dlyhigh,before_avg_dlylow,before_avg_dlynumtrd,before_avg_dlyvol,after_avg_open,after_avg_close,after_avg_dlyhigh,after_avg_dlylow,after_avg_dlynumtrd,after_avg_dlyvol
0,ENDP,2013-05-10,2013-04-19,2013-06-03,36.92,34.970,37.9800,33.99,18771.0,3462429.0,...,37.126667,36.361000,10367.666667,1.991565e+06,34.216000,34.498000,34.950327,33.907667,15575.266667,2.816940e+06
1,ZNGA,2013-06-03,2013-05-10,2013-06-24,3.41,2.991,3.4400,2.87,73590.0,64625735.0,...,3.479667,3.324667,29020.333333,2.190972e+07,2.845333,2.838833,2.892000,2.785333,27614.533333,1.695290e+07
2,MYGN,2013-06-13,2013-05-22,2013-07-05,33.87,32.010,38.2700,30.52,82062.0,16154797.0,...,32.698667,31.681407,6681.200000,9.978447e+05,26.976667,26.871600,27.669933,26.315667,18809.066667,3.487816e+06
3,ALXN,2013-07-12,2013-06-20,2013-08-02,101.51,114.260,125.6500,100.54,63272.0,10871021.0,...,95.125267,92.246907,12706.733333,1.518295e+06,111.749333,111.817333,113.855000,109.718007,17307.800000,2.287365e+06
4,HMNY,2013-08-05,2013-07-15,2013-08-26,3.82,4.520,4.9700,3.82,16.0,8700.0,...,4.485714,4.212857,2.800000,6.611333e+02,5.831340,5.960593,6.468620,5.412673,267.200000,6.396727e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16749,IXHL,2023-12-06,2023-11-30,2023-12-28,8.10,6.770,8.9900,6.51,4007.0,442511.0,...,9.870000,4.552525,7137.750000,6.089890e+05,4.492667,4.342160,4.763807,4.159173,559.200000,5.044713e+04
16750,WBUY,2023-12-06,2023-11-14,2023-12-28,7.45,2.330,14.3000,2.10,61594.0,11174125.0,...,6.091500,5.450000,1093.866667,2.603880e+05,0.916773,0.745687,0.974853,0.686847,8333.533333,2.583257e+06
16751,EFHT,2023-12-06,2023-11-14,2023-12-28,8.50,9.810,12.0000,7.67,568.0,54616.0,...,10.727685,10.567692,54.266667,9.490207e+04,3.385333,2.962667,4.291493,2.602047,3473.533333,4.910621e+05
16752,CHSN,2023-12-06,2023-11-14,2023-12-28,5.20,6.360,6.5000,5.00,23129.0,6193923.0,...,2.277587,1.777300,2138.333333,4.753723e+05,5.424667,5.190667,6.803987,4.196660,11054.533333,2.420138e+06


#### Determine Extremes

In [144]:
nona_results.columns

Index(['ticker', 'current_date', 'before_date', 'after_date', 'open_pr',
       'close_pr', 'dlyhigh', 'dlylow', 'dlynumtrd', 'dlyvol',
       'before_avg_open', 'before_avg_close', 'before_avg_dlyhigh',
       'before_avg_dlylow', 'before_avg_dlynumtrd', 'before_avg_dlyvol',
       'after_avg_open', 'after_avg_close', 'after_avg_dlyhigh',
       'after_avg_dlylow', 'after_avg_dlynumtrd', 'after_avg_dlyvol'],
      dtype='object')

In [145]:
daily_highs = nona_results[["ticker", "current_date", "dlyhigh", "before_avg_dlyhigh",  "after_avg_dlyhigh", "close_pr", "before_avg_close", "after_avg_close"]]
daily_lows = nona_results[["ticker", "current_date", "dlylow", "before_avg_dlylow",  "after_avg_dlylow", "close_pr", "before_avg_close", "after_avg_close"]]

In [150]:
d_highs_fltd = daily_highs[(daily_highs.before_avg_dlyhigh < daily_highs.dlyhigh) \
                           & (daily_highs.after_avg_dlyhigh < daily_highs.dlyhigh)].reset_index(drop=True)
d_lows_fltd = daily_lows[(daily_lows.before_avg_dlylow > daily_lows.dlylow) \
                         & (daily_lows.after_avg_dlylow > daily_lows.dlylow)].reset_index(drop=True)

In [152]:
d_highs_fltd

,ticker,current_date,dlyhigh,before_avg_dlyhigh,after_avg_dlyhigh,close_pr,before_avg_close,after_avg_close
0,ENDP,2013-05-10,37.98,37.126667,34.950327,34.97,36.844000,34.498000
1,MYGN,2013-06-13,38.27,32.698667,27.669933,32.01,32.260667,26.871600
2,ALXN,2013-07-12,125.65,95.125267,113.855000,114.26,93.829467,111.817333
3,HMNY,2013-08-06,8.50,4.460000,6.274440,8.40,4.303067,5.747927
4,HMNY,2013-08-07,9.50,5.044286,6.037773,6.99,4.584400,5.663260
...,...,...,...,...,...,...,...,...
8097,CMND,2023-12-05,6.80,1.812947,3.211740,4.37,1.337660,3.011333
8098,VCXA,2023-12-06,11.00,10.905364,1.748567,9.15,10.175993,1.264387
8099,JAQC,2023-12-06,10.61,10.357793,10.090685,10.60,10.288893,9.161014
8100,WBUY,2023-12-06,14.30,6.091500,0.974853,2.33,5.781667,0.745687


In [147]:
stacked_dedup[stacked_dedup.symbol == 'CMND']

,halt_date,symbol,halt_code,count
50189,2022-11-15,CMND,LUDP,1
50561,2022-12-13,CMND,LUDP,1
50590,2022-12-14,CMND,LUDP,1
50613,2022-12-15,CMND,LUDP,2
50949,2023-01-09,CMND,LUDP,1
52688,2023-05-06,CMND,News pending,2
55200,2023-11-27,CMND,T3,1
55213,2023-11-28,CMND,LUDP,4
55249,2023-11-29,CMND,LUDP,1
55263,2023-11-30,CMND,LUDP,1


In [148]:
daily_lows[(daily_lows.before_avg_dlylow > daily_lows.dlylow) & (daily_lows.after_avg_dlylow > daily_lows.dlylow)]

,ticker,current_date,dlylow,before_avg_dlylow,after_avg_dlylow,close_pr,before_avg_close,after_avg_close
4,HMNY,2013-08-05,3.8200,4.212857,5.412673,4.520,4.292400,5.960593
9,ANDA,2013-08-13,10.0000,10.040000,10.045000,10.040,9.986333,10.055667
12,APSA,2013-08-22,15.9800,18.844753,18.051564,17.800,18.921093,18.192800
14,BGSC,2013-08-23,9.9700,10.067143,10.080000,10.000,10.070000,10.077333
16,CCXI,2013-08-23,6.0500,12.453400,7.226073,8.320,12.608000,7.503467
...,...,...,...,...,...,...,...,...
16717,VIEW,2023-12-01,0.6600,1.790080,2.459333,1.030,1.905393,2.901333
16719,NXTP,2023-12-01,0.2508,0.483837,0.771047,0.625,0.504180,0.963193
16721,SASI,2023-12-01,2.8100,3.262973,2.902667,3.030,3.439533,3.016673
16744,CNET,2023-12-05,0.7600,0.851607,0.929627,1.070,0.880907,0.973847
